In [1]:
import numpy as np
import pandas as pd
import random

import torch
import torch.nn as nn
import torch.nn.functional as F

import seaborn as sns
import matplotlib.pyplot as plt
import os
from datetime import date
import datetime

C:\Users\ladwi\anaconda3\envs\deeplearning\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Generating the training data for the Heat and Diffusion Model

In [2]:
data_dir = "./LakePIAB/"
depth_steps = 25 * 2

print(os.getcwd())

C:\Users\ladwi\Documents\Projects\R\LakePIAB\MCL\01_data_processing


In [3]:
meterological_data_df = pd.read_csv("./../../output/py_meteorology_input.csv")
meterological_data_df = meterological_data_df[1:] # considering everything from 2nd time step

num_time_steps = meterological_data_df.shape[0]
depth_list = np.array(list(range(1, depth_steps+1)) * num_time_steps)
depth_df = pd.DataFrame(data={'depth':depth_list})

#repeating the dataframe depth_steps number of times
meterological_data_df = pd.DataFrame(np.repeat(meterological_data_df.values, depth_steps, axis=0), columns=meterological_data_df.columns)
meterological_data_df = pd.concat([depth_df, meterological_data_df], ignore_index=False, axis=1)
meterological_data_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,icemovAvg,density_snow,ice_prior,snow_prior,snowice_prior,rho_snow_prior,IceSnowAttCoeff_prior,iceFlag_prior,dt_iceon_avg_prior,icemovAvg_prior
0,1,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,5.688714,250.0,0.0,0.0,0.0,250.0,1.0,0.0,0.8,5.688714
1,2,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,5.688714,250.0,0.0,0.0,0.0,250.0,1.0,0.0,0.8,5.688714
2,3,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,5.688714,250.0,0.0,0.0,0.0,250.0,1.0,0.0,0.8,5.688714
3,4,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,5.688714,250.0,0.0,0.0,0.0,250.0,1.0,0.0,0.8,5.688714
4,5,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,5.688714,250.0,0.0,0.0,0.0,250.0,1.0,0.0,0.8,5.688714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1751945,46,2017-12-28 23:00:00,-12.920028,494.226632,-46.98271,-95.220702,0.0,0.4,-999.0,-999.0,...,-0.533471,250.0,0.474353,0.00527,0.0,250.0,0.704183,1.0,0.8,-0.533471
1751946,47,2017-12-28 23:00:00,-12.920028,494.226632,-46.98271,-95.220702,0.0,0.4,-999.0,-999.0,...,-0.533471,250.0,0.474353,0.00527,0.0,250.0,0.704183,1.0,0.8,-0.533471
1751947,48,2017-12-28 23:00:00,-12.920028,494.226632,-46.98271,-95.220702,0.0,0.4,-999.0,-999.0,...,-0.533471,250.0,0.474353,0.00527,0.0,250.0,0.704183,1.0,0.8,-0.533471
1751948,49,2017-12-28 23:00:00,-12.920028,494.226632,-46.98271,-95.220702,0.0,0.4,-999.0,-999.0,...,-0.533471,250.0,0.474353,0.00527,0.0,250.0,0.704183,1.0,0.8,-0.533471


In [4]:
input_temp_df = pd.read_csv("./../../output/py_temp_total05.csv")

flattened_inp_temp = input_temp_df.iloc[:-1,1:].to_numpy().flatten() #this iloc is to remove the time column
flattened_out_temp = input_temp_df.iloc[1:,1:].to_numpy().flatten() #this iloc is to remove the time column
time_stamp = input_temp_df['time'][1:].repeat(depth_steps).values
data = {'time':time_stamp, 'input_temp':flattened_inp_temp, 'temp_total05':flattened_out_temp, 'depth':depth_list}
input_temp_df = pd.DataFrame(data=data)
input_temp_df 

,time,input_temp,temp_total05,depth
0,2013-12-30 01:00:00,3.558955,2.337212,1
1,2013-12-30 01:00:00,4.500009,3.186032,2
2,2013-12-30 01:00:00,4.500005,3.186029,3
3,2013-12-30 01:00:00,4.500005,3.397713,4
4,2013-12-30 01:00:00,4.500005,3.645699,5
...,...,...,...,...
1751945,2017-12-28 23:00:00,4.050675,4.050750,46
1751946,2017-12-28 23:00:00,4.153876,4.153974,47
1751947,2017-12-28 23:00:00,4.261523,4.261658,48
1751948,2017-12-28 23:00:00,4.364835,4.364957,49


In [5]:
depth_list.shape

(1751950,)

In [6]:
final_df = meterological_data_df.merge(input_temp_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,ice_prior,snow_prior,snowice_prior,rho_snow_prior,IceSnowAttCoeff_prior,iceFlag_prior,dt_iceon_avg_prior,icemovAvg_prior,input_temp,temp_total05
0,1,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,0.0,0.0,0.0,250.0,1.0,0.0,0.8,5.688714,3.558955,2.337212
1,2,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,0.0,0.0,0.0,250.0,1.0,0.0,0.8,5.688714,4.500009,3.186032
2,3,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,0.0,0.0,0.0,250.0,1.0,0.0,0.8,5.688714,4.500005,3.186029
3,4,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,0.0,0.0,0.0,250.0,1.0,0.0,0.8,5.688714,4.500005,3.397713
4,5,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,0.0,0.0,0.0,250.0,1.0,0.0,0.8,5.688714,4.500005,3.645699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1751945,46,2017-12-28 23:00:00,-12.920028,494.226632,-46.98271,-95.220702,0.0,0.4,-999.0,-999.0,...,0.474353,0.00527,0.0,250.0,0.704183,1.0,0.8,-0.533471,4.050675,4.050750
1751946,47,2017-12-28 23:00:00,-12.920028,494.226632,-46.98271,-95.220702,0.0,0.4,-999.0,-999.0,...,0.474353,0.00527,0.0,250.0,0.704183,1.0,0.8,-0.533471,4.153876,4.153974
1751947,48,2017-12-28 23:00:00,-12.920028,494.226632,-46.98271,-95.220702,0.0,0.4,-999.0,-999.0,...,0.474353,0.00527,0.0,250.0,0.704183,1.0,0.8,-0.533471,4.261523,4.261658
1751948,49,2017-12-28 23:00:00,-12.920028,494.226632,-46.98271,-95.220702,0.0,0.4,-999.0,-999.0,...,0.474353,0.00527,0.0,250.0,0.704183,1.0,0.8,-0.533471,4.364835,4.364957


In [7]:
buoyancy_data_df = pd.read_csv("./../../output/py_buoyancy.csv")
#Removing Nans
#buoyancy_data_df['n2S-2_1'] = buoyancy_data_df['n2S-2_2']
#buoyancy_data_df['n2S-2_25'] = buoyancy_data_df['n2S-2_24']
buoyancy_data_df['24'] = buoyancy_data_df['23']

buoyancy_data_df = buoyancy_data_df[1:] # considering everything from 2nd time step

flattened_buoy = buoyancy_data_df.iloc[:,1:].to_numpy().flatten()
time_stamp = buoyancy_data_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'buoyancy':flattened_buoy, 'depth':depth_list}
buoyancy_data_df = pd.DataFrame(data=data)

buoyancy_data_df

,time,buoyancy,depth
0,2013-12-30 01:00:00,0.000000,1
1,2013-12-30 01:00:00,0.000074,2
2,2013-12-30 01:00:00,0.000086,3
3,2013-12-30 01:00:00,0.000061,4
4,2013-12-30 01:00:00,0.000062,5
...,...,...,...
1751945,2017-12-28 23:00:00,0.000004,46
1751946,2017-12-28 23:00:00,0.000008,47
1751947,2017-12-28 23:00:00,0.000011,48
1751948,2017-12-28 23:00:00,0.000015,49


In [8]:
final_df = final_df.merge(buoyancy_data_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,snow_prior,snowice_prior,rho_snow_prior,IceSnowAttCoeff_prior,iceFlag_prior,dt_iceon_avg_prior,icemovAvg_prior,input_temp,temp_total05,buoyancy
0,1,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,0.0,0.0,250.0,1.0,0.0,0.8,5.688714,3.558955,2.337212,0.000000
1,2,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,0.0,0.0,250.0,1.0,0.0,0.8,5.688714,4.500009,3.186032,0.000074
2,3,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,0.0,0.0,250.0,1.0,0.0,0.8,5.688714,4.500005,3.186029,0.000086
3,4,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,0.0,0.0,250.0,1.0,0.0,0.8,5.688714,4.500005,3.397713,0.000061
4,5,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,0.0,0.0,250.0,1.0,0.0,0.8,5.688714,4.500005,3.645699,0.000062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1751945,46,2017-12-28 23:00:00,-12.920028,494.226632,-46.98271,-95.220702,0.0,0.4,-999.0,-999.0,...,0.00527,0.0,250.0,0.704183,1.0,0.8,-0.533471,4.050675,4.050750,0.000004
1751946,47,2017-12-28 23:00:00,-12.920028,494.226632,-46.98271,-95.220702,0.0,0.4,-999.0,-999.0,...,0.00527,0.0,250.0,0.704183,1.0,0.8,-0.533471,4.153876,4.153974,0.000008
1751947,48,2017-12-28 23:00:00,-12.920028,494.226632,-46.98271,-95.220702,0.0,0.4,-999.0,-999.0,...,0.00527,0.0,250.0,0.704183,1.0,0.8,-0.533471,4.261523,4.261658,0.000011
1751948,49,2017-12-28 23:00:00,-12.920028,494.226632,-46.98271,-95.220702,0.0,0.4,-999.0,-999.0,...,0.00527,0.0,250.0,0.704183,1.0,0.8,-0.533471,4.364835,4.364957,0.000015


In [9]:
out_diffusivity_df = pd.read_csv("./../../output/py_diff.csv")

out_diffusivity_df = out_diffusivity_df[1:] # considering everything from 2nd time step

flattened_diff = out_diffusivity_df.iloc[:,1:].to_numpy().flatten()
time_stamp = out_diffusivity_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'diffusivity':flattened_diff, 'depth':depth_list}
out_diffusivity_df = pd.DataFrame(data=data)

out_diffusivity_df

,time,diffusivity,depth
0,2013-12-30 01:00:00,3.297723e+00,1
1,2013-12-30 01:00:00,3.297723e+00,2
2,2013-12-30 01:00:00,1.848158e+00,3
3,2013-12-30 01:00:00,7.018665e-01,4
4,2013-12-30 01:00:00,2.734765e-01,5
...,...,...,...
1751945,2017-12-28 23:00:00,2.800000e-07,46
1751946,2017-12-28 23:00:00,2.800000e-07,47
1751947,2017-12-28 23:00:00,2.800000e-07,48
1751948,2017-12-28 23:00:00,2.800000e-07,49


In [10]:
final_df = final_df.merge(out_diffusivity_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,snowice_prior,rho_snow_prior,IceSnowAttCoeff_prior,iceFlag_prior,dt_iceon_avg_prior,icemovAvg_prior,input_temp,temp_total05,buoyancy,diffusivity
0,1,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,0.0,250.0,1.0,0.0,0.8,5.688714,3.558955,2.337212,0.000000,3.297723e+00
1,2,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,0.0,250.0,1.0,0.0,0.8,5.688714,4.500009,3.186032,0.000074,3.297723e+00
2,3,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,0.0,250.0,1.0,0.0,0.8,5.688714,4.500005,3.186029,0.000086,1.848158e+00
3,4,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,0.0,250.0,1.0,0.0,0.8,5.688714,4.500005,3.397713,0.000061,7.018665e-01
4,5,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,0.0,250.0,1.0,0.0,0.8,5.688714,4.500005,3.645699,0.000062,2.734765e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1751945,46,2017-12-28 23:00:00,-12.920028,494.226632,-46.98271,-95.220702,0.0,0.4,-999.0,-999.0,...,0.0,250.0,0.704183,1.0,0.8,-0.533471,4.050675,4.050750,0.000004,2.800000e-07
1751946,47,2017-12-28 23:00:00,-12.920028,494.226632,-46.98271,-95.220702,0.0,0.4,-999.0,-999.0,...,0.0,250.0,0.704183,1.0,0.8,-0.533471,4.153876,4.153974,0.000008,2.800000e-07
1751947,48,2017-12-28 23:00:00,-12.920028,494.226632,-46.98271,-95.220702,0.0,0.4,-999.0,-999.0,...,0.0,250.0,0.704183,1.0,0.8,-0.533471,4.261523,4.261658,0.000011,2.800000e-07
1751948,49,2017-12-28 23:00:00,-12.920028,494.226632,-46.98271,-95.220702,0.0,0.4,-999.0,-999.0,...,0.0,250.0,0.704183,1.0,0.8,-0.533471,4.364835,4.364957,0.000015,2.800000e-07


In [11]:
out_temp_df = pd.read_csv("./../../output/py_temp_heat01.csv")

out_temp_df = out_temp_df[1:] # considering everything from 2nd time step

flattened_temp = out_temp_df.iloc[:,1:].to_numpy().flatten()
time_stamp = out_temp_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'temp_heat01':flattened_temp, 'depth':depth_list}
out_temp_df = pd.DataFrame(data=data)

out_temp_df

,time,temp_heat01,depth
0,2013-12-30 01:00:00,2.337212,1
1,2013-12-30 01:00:00,3.186032,2
2,2013-12-30 01:00:00,3.186029,3
3,2013-12-30 01:00:00,3.397713,4
4,2013-12-30 01:00:00,3.645699,5
...,...,...,...
1751945,2017-12-28 23:00:00,4.050750,46
1751946,2017-12-28 23:00:00,4.153974,47
1751947,2017-12-28 23:00:00,4.261658,48
1751948,2017-12-28 23:00:00,4.364957,49


In [12]:
final_df = final_df.merge(out_temp_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,rho_snow_prior,IceSnowAttCoeff_prior,iceFlag_prior,dt_iceon_avg_prior,icemovAvg_prior,input_temp,temp_total05,buoyancy,diffusivity,temp_heat01
0,1,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,250.0,1.0,0.0,0.8,5.688714,3.558955,2.337212,0.000000,3.297723e+00,2.337212
1,2,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,250.0,1.0,0.0,0.8,5.688714,4.500009,3.186032,0.000074,3.297723e+00,3.186032
2,3,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,250.0,1.0,0.0,0.8,5.688714,4.500005,3.186029,0.000086,1.848158e+00,3.186029
3,4,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,250.0,1.0,0.0,0.8,5.688714,4.500005,3.397713,0.000061,7.018665e-01,3.397713
4,5,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,250.0,1.0,0.0,0.8,5.688714,4.500005,3.645699,0.000062,2.734765e-01,3.645699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1751945,46,2017-12-28 23:00:00,-12.920028,494.226632,-46.98271,-95.220702,0.0,0.4,-999.0,-999.0,...,250.0,0.704183,1.0,0.8,-0.533471,4.050675,4.050750,0.000004,2.800000e-07,4.050750
1751946,47,2017-12-28 23:00:00,-12.920028,494.226632,-46.98271,-95.220702,0.0,0.4,-999.0,-999.0,...,250.0,0.704183,1.0,0.8,-0.533471,4.153876,4.153974,0.000008,2.800000e-07,4.153974
1751947,48,2017-12-28 23:00:00,-12.920028,494.226632,-46.98271,-95.220702,0.0,0.4,-999.0,-999.0,...,250.0,0.704183,1.0,0.8,-0.533471,4.261523,4.261658,0.000011,2.800000e-07,4.261658
1751948,49,2017-12-28 23:00:00,-12.920028,494.226632,-46.98271,-95.220702,0.0,0.4,-999.0,-999.0,...,250.0,0.704183,1.0,0.8,-0.533471,4.364835,4.364957,0.000015,2.800000e-07,4.364957


In [13]:
out_temp_df = pd.read_csv("./../../output/py_temp_diff02.csv")

out_temp_df = out_temp_df[1:] # considering everything from 2nd time step

flattened_temp = out_temp_df.iloc[:,1:].to_numpy().flatten()
time_stamp = out_temp_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'temp_diff02':flattened_temp, 'depth':depth_list}
out_temp_df = pd.DataFrame(data=data)

out_temp_df

,time,temp_diff02,depth
0,2013-12-30 01:00:00,2.337212,1
1,2013-12-30 01:00:00,1.860031,2
2,2013-12-30 01:00:00,2.231620,3
3,2013-12-30 01:00:00,2.391449,4
4,2013-12-30 01:00:00,2.514778,5
...,...,...,...
1751945,2017-12-28 23:00:00,4.050760,46
1751946,2017-12-28 23:00:00,4.153992,47
1751947,2017-12-28 23:00:00,4.261641,48
1751948,2017-12-28 23:00:00,4.364993,49


In [14]:
final_df = final_df.merge(out_temp_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,IceSnowAttCoeff_prior,iceFlag_prior,dt_iceon_avg_prior,icemovAvg_prior,input_temp,temp_total05,buoyancy,diffusivity,temp_heat01,temp_diff02
0,1,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,1.0,0.0,0.8,5.688714,3.558955,2.337212,0.000000,3.297723e+00,2.337212,2.337212
1,2,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,1.0,0.0,0.8,5.688714,4.500009,3.186032,0.000074,3.297723e+00,3.186032,1.860031
2,3,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,1.0,0.0,0.8,5.688714,4.500005,3.186029,0.000086,1.848158e+00,3.186029,2.231620
3,4,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,1.0,0.0,0.8,5.688714,4.500005,3.397713,0.000061,7.018665e-01,3.397713,2.391449
4,5,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,1.0,0.0,0.8,5.688714,4.500005,3.645699,0.000062,2.734765e-01,3.645699,2.514778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1751945,46,2017-12-28 23:00:00,-12.920028,494.226632,-46.98271,-95.220702,0.0,0.4,-999.0,-999.0,...,0.704183,1.0,0.8,-0.533471,4.050675,4.050750,0.000004,2.800000e-07,4.050750,4.050760
1751946,47,2017-12-28 23:00:00,-12.920028,494.226632,-46.98271,-95.220702,0.0,0.4,-999.0,-999.0,...,0.704183,1.0,0.8,-0.533471,4.153876,4.153974,0.000008,2.800000e-07,4.153974,4.153992
1751947,48,2017-12-28 23:00:00,-12.920028,494.226632,-46.98271,-95.220702,0.0,0.4,-999.0,-999.0,...,0.704183,1.0,0.8,-0.533471,4.261523,4.261658,0.000011,2.800000e-07,4.261658,4.261641
1751948,49,2017-12-28 23:00:00,-12.920028,494.226632,-46.98271,-95.220702,0.0,0.4,-999.0,-999.0,...,0.704183,1.0,0.8,-0.533471,4.364835,4.364957,0.000015,2.800000e-07,4.364957,4.364993


In [15]:
datetime_list =[datetime.datetime.strptime(date, '%Y-%m-%d %H:%M:%S') for date in final_df['time']]
day_of_year_list = [t.timetuple().tm_yday for t in datetime_list]
time_of_day_list = [t.hour for t in datetime_list]

In [16]:
final_df['day_of_year']=day_of_year_list
final_df['time_of_day']=time_of_day_list

In [17]:
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,dt_iceon_avg_prior,icemovAvg_prior,input_temp,temp_total05,buoyancy,diffusivity,temp_heat01,temp_diff02,day_of_year,time_of_day
0,1,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,0.8,5.688714,3.558955,2.337212,0.000000,3.297723e+00,2.337212,2.337212,364,1
1,2,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,0.8,5.688714,4.500009,3.186032,0.000074,3.297723e+00,3.186032,1.860031,364,1
2,3,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,0.8,5.688714,4.500005,3.186029,0.000086,1.848158e+00,3.186029,2.231620,364,1
3,4,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,0.8,5.688714,4.500005,3.397713,0.000061,7.018665e-01,3.397713,2.391449,364,1
4,5,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,0.8,5.688714,4.500005,3.645699,0.000062,2.734765e-01,3.645699,2.514778,364,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1751945,46,2017-12-28 23:00:00,-12.920028,494.226632,-46.98271,-95.220702,0.0,0.4,-999.0,-999.0,...,0.8,-0.533471,4.050675,4.050750,0.000004,2.800000e-07,4.050750,4.050760,362,23
1751946,47,2017-12-28 23:00:00,-12.920028,494.226632,-46.98271,-95.220702,0.0,0.4,-999.0,-999.0,...,0.8,-0.533471,4.153876,4.153974,0.000008,2.800000e-07,4.153974,4.153992,362,23
1751947,48,2017-12-28 23:00:00,-12.920028,494.226632,-46.98271,-95.220702,0.0,0.4,-999.0,-999.0,...,0.8,-0.533471,4.261523,4.261658,0.000011,2.800000e-07,4.261658,4.261641,362,23
1751948,49,2017-12-28 23:00:00,-12.920028,494.226632,-46.98271,-95.220702,0.0,0.4,-999.0,-999.0,...,0.8,-0.533471,4.364835,4.364957,0.000015,2.800000e-07,4.364957,4.364993,362,23


In [18]:
temp_mix03_df = pd.read_csv("./../../output/py_temp_mix03.csv")

temp_mix03_df = temp_mix03_df[1:] # considering everything from 2nd time step

flattened_temp = temp_mix03_df.iloc[:,1:].to_numpy().flatten()
time_stamp = temp_mix03_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'temp_mix03':flattened_temp, 'depth':depth_list}
temp_mix03_df = pd.DataFrame(data=data)

temp_mix03_df

,time,temp_mix03,depth
0,2013-12-30 01:00:00,2.337212,1
1,2013-12-30 01:00:00,1.860031,2
2,2013-12-30 01:00:00,2.231620,3
3,2013-12-30 01:00:00,2.391449,4
4,2013-12-30 01:00:00,2.514778,5
...,...,...,...
1751945,2017-12-28 23:00:00,4.050760,46
1751946,2017-12-28 23:00:00,4.153992,47
1751947,2017-12-28 23:00:00,4.261641,48
1751948,2017-12-28 23:00:00,4.364993,49


In [19]:
final_df = final_df.merge(temp_mix03_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,icemovAvg_prior,input_temp,temp_total05,buoyancy,diffusivity,temp_heat01,temp_diff02,day_of_year,time_of_day,temp_mix03
0,1,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,5.688714,3.558955,2.337212,0.000000,3.297723e+00,2.337212,2.337212,364,1,2.337212
1,2,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,5.688714,4.500009,3.186032,0.000074,3.297723e+00,3.186032,1.860031,364,1,1.860031
2,3,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,5.688714,4.500005,3.186029,0.000086,1.848158e+00,3.186029,2.231620,364,1,2.231620
3,4,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,5.688714,4.500005,3.397713,0.000061,7.018665e-01,3.397713,2.391449,364,1,2.391449
4,5,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,5.688714,4.500005,3.645699,0.000062,2.734765e-01,3.645699,2.514778,364,1,2.514778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1751945,46,2017-12-28 23:00:00,-12.920028,494.226632,-46.98271,-95.220702,0.0,0.4,-999.0,-999.0,...,-0.533471,4.050675,4.050750,0.000004,2.800000e-07,4.050750,4.050760,362,23,4.050760
1751946,47,2017-12-28 23:00:00,-12.920028,494.226632,-46.98271,-95.220702,0.0,0.4,-999.0,-999.0,...,-0.533471,4.153876,4.153974,0.000008,2.800000e-07,4.153974,4.153992,362,23,4.153992
1751947,48,2017-12-28 23:00:00,-12.920028,494.226632,-46.98271,-95.220702,0.0,0.4,-999.0,-999.0,...,-0.533471,4.261523,4.261658,0.000011,2.800000e-07,4.261658,4.261641,362,23,4.261641
1751948,49,2017-12-28 23:00:00,-12.920028,494.226632,-46.98271,-95.220702,0.0,0.4,-999.0,-999.0,...,-0.533471,4.364835,4.364957,0.000015,2.800000e-07,4.364957,4.364993,362,23,4.364993


In [20]:
temp_conv03_df = pd.read_csv("./../../output/py_temp_conv04.csv")

temp_conv03_df = temp_conv03_df[1:] # considering everything from 2nd time step

flattened_temp = temp_conv03_df.iloc[:,1:].to_numpy().flatten()
time_stamp = temp_conv03_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'temp_conv04':flattened_temp, 'depth':depth_list}
temp_conv03_df = pd.DataFrame(data=data)

temp_conv03_df

,time,temp_conv04,depth
0,2013-12-30 01:00:00,2.104677,1
1,2013-12-30 01:00:00,2.104677,2
2,2013-12-30 01:00:00,2.231620,3
3,2013-12-30 01:00:00,2.391449,4
4,2013-12-30 01:00:00,2.514778,5
...,...,...,...
1751945,2017-12-28 23:00:00,4.050760,46
1751946,2017-12-28 23:00:00,4.153992,47
1751947,2017-12-28 23:00:00,4.261641,48
1751948,2017-12-28 23:00:00,4.364993,49


In [21]:
final_df = final_df.merge(temp_conv03_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,input_temp,temp_total05,buoyancy,diffusivity,temp_heat01,temp_diff02,day_of_year,time_of_day,temp_mix03,temp_conv04
0,1,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,3.558955,2.337212,0.000000,3.297723e+00,2.337212,2.337212,364,1,2.337212,2.104677
1,2,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,4.500009,3.186032,0.000074,3.297723e+00,3.186032,1.860031,364,1,1.860031,2.104677
2,3,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,4.500005,3.186029,0.000086,1.848158e+00,3.186029,2.231620,364,1,2.231620,2.231620
3,4,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,4.500005,3.397713,0.000061,7.018665e-01,3.397713,2.391449,364,1,2.391449,2.391449
4,5,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,4.500005,3.645699,0.000062,2.734765e-01,3.645699,2.514778,364,1,2.514778,2.514778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1751945,46,2017-12-28 23:00:00,-12.920028,494.226632,-46.98271,-95.220702,0.0,0.4,-999.0,-999.0,...,4.050675,4.050750,0.000004,2.800000e-07,4.050750,4.050760,362,23,4.050760,4.050760
1751946,47,2017-12-28 23:00:00,-12.920028,494.226632,-46.98271,-95.220702,0.0,0.4,-999.0,-999.0,...,4.153876,4.153974,0.000008,2.800000e-07,4.153974,4.153992,362,23,4.153992,4.153992
1751947,48,2017-12-28 23:00:00,-12.920028,494.226632,-46.98271,-95.220702,0.0,0.4,-999.0,-999.0,...,4.261523,4.261658,0.000011,2.800000e-07,4.261658,4.261641,362,23,4.261641,4.261641
1751948,49,2017-12-28 23:00:00,-12.920028,494.226632,-46.98271,-95.220702,0.0,0.4,-999.0,-999.0,...,4.364835,4.364957,0.000015,2.800000e-07,4.364957,4.364993,362,23,4.364993,4.364993


In [22]:
temp_initial00_df = pd.read_csv("./../../output/py_temp_initial00.csv")

temp_initial00_df = temp_initial00_df[1:] # considering everything from 2nd time step

flattened_temp = temp_initial00_df.iloc[:,1:].to_numpy().flatten()
time_stamp = temp_initial00_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'temp_initial00':flattened_temp, 'depth':depth_list}
temp_initial00_df = pd.DataFrame(data=data)

temp_initial00_df

,time,temp_initial00,depth
0,2013-12-30 01:00:00,3.228721,1
1,2013-12-30 01:00:00,3.186024,2
2,2013-12-30 01:00:00,3.186024,3
3,2013-12-30 01:00:00,3.397708,4
4,2013-12-30 01:00:00,3.645694,5
...,...,...,...
1751945,2017-12-28 23:00:00,4.050685,46
1751946,2017-12-28 23:00:00,4.153894,47
1751947,2017-12-28 23:00:00,4.261506,48
1751948,2017-12-28 23:00:00,4.364871,49


In [23]:
final_df = final_df.merge(temp_initial00_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,temp_total05,buoyancy,diffusivity,temp_heat01,temp_diff02,day_of_year,time_of_day,temp_mix03,temp_conv04,temp_initial00
0,1,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,2.337212,0.000000,3.297723e+00,2.337212,2.337212,364,1,2.337212,2.104677,3.228721
1,2,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,3.186032,0.000074,3.297723e+00,3.186032,1.860031,364,1,1.860031,2.104677,3.186024
2,3,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,3.186029,0.000086,1.848158e+00,3.186029,2.231620,364,1,2.231620,2.231620,3.186024
3,4,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,3.397713,0.000061,7.018665e-01,3.397713,2.391449,364,1,2.391449,2.391449,3.397708
4,5,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,3.645699,0.000062,2.734765e-01,3.645699,2.514778,364,1,2.514778,2.514778,3.645694
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1751945,46,2017-12-28 23:00:00,-12.920028,494.226632,-46.98271,-95.220702,0.0,0.4,-999.0,-999.0,...,4.050750,0.000004,2.800000e-07,4.050750,4.050760,362,23,4.050760,4.050760,4.050685
1751946,47,2017-12-28 23:00:00,-12.920028,494.226632,-46.98271,-95.220702,0.0,0.4,-999.0,-999.0,...,4.153974,0.000008,2.800000e-07,4.153974,4.153992,362,23,4.153992,4.153992,4.153894
1751947,48,2017-12-28 23:00:00,-12.920028,494.226632,-46.98271,-95.220702,0.0,0.4,-999.0,-999.0,...,4.261658,0.000011,2.800000e-07,4.261658,4.261641,362,23,4.261641,4.261641,4.261506
1751948,49,2017-12-28 23:00:00,-12.920028,494.226632,-46.98271,-95.220702,0.0,0.4,-999.0,-999.0,...,4.364957,0.000015,2.800000e-07,4.364957,4.364993,362,23,4.364993,4.364993,4.364871


In [24]:
temp_obs_df = pd.read_csv("./../../output/py_observed_temp.csv")

temp_obs_df = temp_obs_df[1:] # considering everything from 2nd time step

flattened_temp = temp_obs_df.iloc[:,1:].to_numpy().flatten()
time_stamp = temp_obs_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'obs_temp':flattened_temp, 'depth':depth_list}

print(data)
print(len(data['time']))
print(len(data['obs_temp']))
print(len(data['depth']))

temp_obs_df = pd.DataFrame(data=data)

temp_obs_df

print(flattened_temp.shape)

print(time_stamp.shape)

print(depth_list.shape)

{'time': array(['2013-12-30 01:00:00', '2013-12-30 01:00:00',
       '2013-12-30 01:00:00', ..., '2017-12-28 23:00:00',
       '2017-12-28 23:00:00', '2017-12-28 23:00:00'], dtype=object), 'obs_temp': array([2.5327381 , 2.70760582, 2.70760582, ..., 4.00578453, 4.82646421,
       7.99623398]), 'depth': array([ 1,  2,  3, ..., 48, 49, 50])}
1751950
1751950
1751950
(1751950,)
(1751950,)
(1751950,)


In [25]:
final_df = final_df.merge(temp_obs_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,buoyancy,diffusivity,temp_heat01,temp_diff02,day_of_year,time_of_day,temp_mix03,temp_conv04,temp_initial00,obs_temp
0,1,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,0.000000,3.297723e+00,2.337212,2.337212,364,1,2.337212,2.104677,3.228721,2.532738
1,2,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,0.000074,3.297723e+00,3.186032,1.860031,364,1,1.860031,2.104677,3.186024,2.707606
2,3,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,0.000086,1.848158e+00,3.186029,2.231620,364,1,2.231620,2.231620,3.186024,2.707606
3,4,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,0.000061,7.018665e-01,3.397713,2.391449,364,1,2.391449,2.391449,3.397708,2.707606
4,5,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,0.000062,2.734765e-01,3.645699,2.514778,364,1,2.514778,2.514778,3.645694,2.707606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1751945,46,2017-12-28 23:00:00,-12.920028,494.226632,-46.98271,-95.220702,0.0,0.4,-999.0,-999.0,...,0.000004,2.800000e-07,4.050750,4.050760,362,23,4.050760,4.050760,4.050685,3.029762
1751946,47,2017-12-28 23:00:00,-12.920028,494.226632,-46.98271,-95.220702,0.0,0.4,-999.0,-999.0,...,0.000008,2.800000e-07,4.153974,4.153992,362,23,4.153992,4.153992,4.153894,3.185105
1751947,48,2017-12-28 23:00:00,-12.920028,494.226632,-46.98271,-95.220702,0.0,0.4,-999.0,-999.0,...,0.000011,2.800000e-07,4.261658,4.261641,362,23,4.261641,4.261641,4.261506,4.005785
1751948,49,2017-12-28 23:00:00,-12.920028,494.226632,-46.98271,-95.220702,0.0,0.4,-999.0,-999.0,...,0.000015,2.800000e-07,4.364957,4.364993,362,23,4.364993,4.364993,4.364871,4.826464


In [26]:

#temp_obs_df2 = pd.read_csv("./../output/observed_temp.csv")
#temp_obs_df = pd.read_csv("./../output/observed_temp.csv")


#print(flattened_inp_temp_obs.shape)

#temp_obs_df2 = temp_obs_df2[:-1] # considering everything from 2nd time step

#flattened_inp_temp_obs = temp_obs_df2.iloc[:,1:].to_numpy().flatten()

#time_stamp = input_temp_df['time'][1:].repeat(depth_steps).values

#data = {'time':time_stamp, 'obs_input':flattened_inp_temp_obs, 'depth':depth_list}
#temp_obs_df2 = pd.DataFrame(data=data)

#temp_obs_df2


temp_obs_df = pd.read_csv("./../../output/py_observed_temp.csv")

flattened_inp_temp = temp_obs_df.iloc[:-1,1:].to_numpy().flatten() #this iloc is to remove the time column
print(flattened_inp_temp.shape)


time_stamp = temp_obs_df['time'][1:].repeat(depth_steps).values
print(time_stamp.shape)

print(depth_list.shape)
data = {'time':time_stamp, 'input_obs':flattened_inp_temp,'depth':depth_list}
temp_obs_df = pd.DataFrame(data=data)
temp_obs_df 

(1751950,)
(1751950,)
(1751950,)


,time,input_obs,depth
0,2013-12-30 01:00:00,2.535714,1
1,2013-12-30 01:00:00,2.710317,2
2,2013-12-30 01:00:00,2.710317,3
3,2013-12-30 01:00:00,2.710317,4
4,2013-12-30 01:00:00,2.710317,5
...,...,...,...
1751945,2017-12-28 23:00:00,3.030952,46
1751946,2017-12-28 23:00:00,3.189226,47
1751947,2017-12-28 23:00:00,4.009906,48
1751948,2017-12-28 23:00:00,4.830586,49


In [27]:
final_df = final_df.merge(temp_obs_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,diffusivity,temp_heat01,temp_diff02,day_of_year,time_of_day,temp_mix03,temp_conv04,temp_initial00,obs_temp,input_obs
0,1,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,3.297723e+00,2.337212,2.337212,364,1,2.337212,2.104677,3.228721,2.532738,2.535714
1,2,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,3.297723e+00,3.186032,1.860031,364,1,1.860031,2.104677,3.186024,2.707606,2.710317
2,3,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,1.848158e+00,3.186029,2.231620,364,1,2.231620,2.231620,3.186024,2.707606,2.710317
3,4,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,7.018665e-01,3.397713,2.391449,364,1,2.391449,2.391449,3.397708,2.707606,2.710317
4,5,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,2.734765e-01,3.645699,2.514778,364,1,2.514778,2.514778,3.645694,2.707606,2.710317
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1752345,46,2017-12-28 23:00:00,-12.920028,494.226632,-46.98271,-95.220702,0.0,0.4,-999.0,-999.0,...,2.800000e-07,4.050750,4.050760,362,23,4.050760,4.050760,4.050685,3.029762,3.030952
1752346,47,2017-12-28 23:00:00,-12.920028,494.226632,-46.98271,-95.220702,0.0,0.4,-999.0,-999.0,...,2.800000e-07,4.153974,4.153992,362,23,4.153992,4.153992,4.153894,3.185105,3.189226
1752347,48,2017-12-28 23:00:00,-12.920028,494.226632,-46.98271,-95.220702,0.0,0.4,-999.0,-999.0,...,2.800000e-07,4.261658,4.261641,362,23,4.261641,4.261641,4.261506,4.005785,4.009906
1752348,49,2017-12-28 23:00:00,-12.920028,494.226632,-46.98271,-95.220702,0.0,0.4,-999.0,-999.0,...,2.800000e-07,4.364957,4.364993,362,23,4.364993,4.364993,4.364871,4.826464,4.830586


In [28]:
ice_data_df = pd.read_csv("./../../output/py_icesnow.csv")
ice_data_df = ice_data_df[1:] # considering everything from 2nd time step

num_time_steps = ice_data_df.shape[0]
depth_list = np.array(list(range(1, depth_steps+1)) * num_time_steps)
depth_df = pd.DataFrame(data={'depth':depth_list})

#repeating the dataframe depth_steps number of times
ice_data_df = pd.DataFrame(np.repeat(ice_data_df.values, depth_steps, axis=0), columns=ice_data_df.columns)
ice_data_df = pd.concat([depth_df, ice_data_df], ignore_index=False, axis=1)
print(ice_data_df)

final_df = final_df.merge(ice_data_df, how='inner', on=['time','depth'])
final_df

         depth                 time       ice     snow snowice
0            1  2013-12-30 01:00:00       0.0      0.0     0.0
1            2  2013-12-30 01:00:00       0.0      0.0     0.0
2            3  2013-12-30 01:00:00       0.0      0.0     0.0
3            4  2013-12-30 01:00:00       0.0      0.0     0.0
4            5  2013-12-30 01:00:00       0.0      0.0     0.0
...        ...                  ...       ...      ...     ...
1751945     46  2017-12-28 23:00:00  0.474353  0.00527     0.0
1751946     47  2017-12-28 23:00:00  0.474353  0.00527     0.0
1751947     48  2017-12-28 23:00:00  0.474353  0.00527     0.0
1751948     49  2017-12-28 23:00:00  0.474353  0.00527     0.0
1751949     50  2017-12-28 23:00:00  0.474353  0.00527     0.0

[1751950 rows x 5 columns]


,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,day_of_year,time_of_day,temp_mix03,temp_conv04,temp_initial00,obs_temp,input_obs,ice,snow,snowice
0,1,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,364,1,2.337212,2.104677,3.228721,2.532738,2.535714,0.0,0.0,0.0
1,2,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,364,1,1.860031,2.104677,3.186024,2.707606,2.710317,0.0,0.0,0.0
2,3,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,364,1,2.231620,2.231620,3.186024,2.707606,2.710317,0.0,0.0,0.0
3,4,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,364,1,2.391449,2.391449,3.397708,2.707606,2.710317,0.0,0.0,0.0
4,5,2013-12-30 01:00:00,-19.719746,471.661536,-101.200398,-231.960902,0.0,0.4,-999.0,-999.0,...,364,1,2.514778,2.514778,3.645694,2.707606,2.710317,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1752345,46,2017-12-28 23:00:00,-12.920028,494.226632,-46.98271,-95.220702,0.0,0.4,-999.0,-999.0,...,362,23,4.050760,4.050760,4.050685,3.029762,3.030952,0.474353,0.00527,0.0
1752346,47,2017-12-28 23:00:00,-12.920028,494.226632,-46.98271,-95.220702,0.0,0.4,-999.0,-999.0,...,362,23,4.153992,4.153992,4.153894,3.185105,3.189226,0.474353,0.00527,0.0
1752347,48,2017-12-28 23:00:00,-12.920028,494.226632,-46.98271,-95.220702,0.0,0.4,-999.0,-999.0,...,362,23,4.261641,4.261641,4.261506,4.005785,4.009906,0.474353,0.00527,0.0
1752348,49,2017-12-28 23:00:00,-12.920028,494.226632,-46.98271,-95.220702,0.0,0.4,-999.0,-999.0,...,362,23,4.364993,4.364993,4.364871,4.826464,4.830586,0.474353,0.00527,0.0


In [29]:
obs_array = final_df['obs_temp']
obs_array[obs_array == -999] = final_df['temp_total05']
print(obs_array)
final_df['obs_temp'] = obs_array

0          2.532738
1          2.707606
2          2.707606
3          2.707606
4          2.707606
             ...   
1752345    3.029762
1752346    3.185105
1752347    4.005785
1752348    4.826464
1752349    7.996234
Name: obs_temp, Length: 1752350, dtype: float64


C:\Users\ladwi\AppData\Local\Temp\ipykernel_7524\1668945599.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  obs_array[obs_array == -999] = final_df['temp_total05']


In [30]:
final_df_null = final_df.fillna('')
print(final_df_null.head)

<bound method NDFrame.head of          depth                 time  AirTemp_degC  Longwave_Wm-2  Latent_Wm-2  \
0            1  2013-12-30 01:00:00    -19.719746     471.661536  -101.200398   
1            2  2013-12-30 01:00:00    -19.719746     471.661536  -101.200398   
2            3  2013-12-30 01:00:00    -19.719746     471.661536  -101.200398   
3            4  2013-12-30 01:00:00    -19.719746     471.661536  -101.200398   
4            5  2013-12-30 01:00:00    -19.719746     471.661536  -101.200398   
...        ...                  ...           ...            ...          ...   
1752345     46  2017-12-28 23:00:00    -12.920028     494.226632   -46.982710   
1752346     47  2017-12-28 23:00:00    -12.920028     494.226632   -46.982710   
1752347     48  2017-12-28 23:00:00    -12.920028     494.226632   -46.982710   
1752348     49  2017-12-28 23:00:00    -12.920028     494.226632   -46.982710   
1752349     50  2017-12-28 23:00:00    -12.920028     494.226632   -46.982710  

In [31]:
final_df_null.to_csv("./../02_training/all_data_lake_modeling_in_time.csv", index=False)

In [32]:
final_df_null.to_csv("all_data_lake_modeling_in_time.csv", index=False)